# 1. Protein

## 1.0 Startup

In [1]:
disp ('executing gpml startup script...')
mydir = fileparts (mfilename ('fullpath'));                 % where am I located
addpath (mydir)
dirs = {'cov','doc','inf','lik','mean','prior','util'};           % core folders
for d = dirs, addpath (fullfile (mydir, d{1})), end
dirs = {{'util','minfunc'},{'util','minfunc','compiled'}};     % minfunc folders
for d = dirs, addpath (fullfile (mydir, d{1}{:})), end
% addpath([mydir,'/util/sparseinv'])
pkg load statistics

executing gpml startup script...


In [2]:
dataset = 'protein'
load ('./data/protein.mat');
[ndata, ~] = size(M);
X = M(:,2:9);
Y = M(:,1);
R = randperm(ndata);        
xt = X(R(1:5730),:);
yt = Y(R(1:5730));
R(1:5730) = [];
x = X(R,:);          
y = Y(R);
n = 40000; nt = 5730;   %Ms = 50

dataset = protein


In [3]:
sf2 = 1 ; ell = 1 ; sn2 = 0.1 ; 
d = size(x,2);
hyp.cov = log([ones(d,1)*ell;sf2]); hyp.lik = log(sn2); hyp.mean = [];
opts.Xnorm = 'Y' ; opts.Ynorm = 'Y' ;
opts.Ms = 50 ;
opts.ell = ell ; opts.sf2 = sf2 ; opts.sn2 = sn2 ;
opts.meanfunc = []; opts.covfunc = @covSEard; opts.likfunc = @likGauss; opts.inffunc = @infGaussLik ;
opts.numOptFC = 25 ;

partitionCriterion = 'kmeans';
opts.partitionCriterion = partitionCriterion ;
[models,t_dGP_train] = aggregation_train(x,y,opts) ;
save('./res/proteinmodel', 'models','t_dGP_train')

In [4]:
partitionCriterion = 'kmeans'; %, 'kmeans', 'knkmeans'
opts.numOptFC = 25 ;
opts.partitionCriterion = partitionCriterion ;
s = load('./res/proteinmodel');
models = s.models;


In [5]:
criterions = {'BCM','PoE'}; %, 'GPoE', 'KLRBCM', 'KLGRBCM', 'GRBCM'

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
    filename = sprintf('./res/%s_%s_%s.mat', dataset, partitionCriterion, criterion);
    save(filename, 'SMSE','MSLL','NLPD')
endfor

Processing protein kmeans BCM
BCM: SMSE 6.03160283, MSLL -0.8188, NLPD 2.4155
Processing protein kmeans PoE
PoE: SMSE 29.66232146, MSLL 18.0887, NLPD 21.3230


## 1.1 RBCM vs TERBCM

### 1.1.1 RBCM baseline

In [6]:
criterions = {'RBCM'}; %, 'GPoE', 'KLRBCM', 'KLGRBCM', 'GRBCM'

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
    filename = sprintf('./res/%s_%s_%s.mat', dataset, partitionCriterion, criterion);
    save(filename, 'SMSE','MSLL','NLPD')
endfor

Processing protein kmeans RBCM
RBCM: SMSE 5.31678021, MSLL -0.9648, NLPD 2.2695


### 1.1.2 Grid q

In [7]:
partitionCriterions = {'kmeans'}; % 'random', 'kmeans', 'knkmeans'
criterions = {'TERBCM'}; %, 'TEGRBCM' 'TEGPoE'};
qs = 0.2:0.2:3.0;

for i = 1:length(partitionCriterions)
    partitionCriterion = partitionCriterions{i};
    for j = 1:length(criterions)     
        criterion = criterions{j};
        printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
        opts.partitionCriterion = partitionCriterion ;
        %[models,t_dGP_train] = aggregation_train(x,y,opts) ;
        for q = qs
            [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
            [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
            printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f - q %6.2f\r\n', criterion, SMSE,MSLL,NLPD,q);
            filename = sprintf('./res/terbcm%s_%s_%s_%6.2f.mat', dataset, partitionCriterion, criterion, q);
            save(filename, 'SMSE','MSLL','NLPD')
        endfor
    endfor
endfor


Processing protein kmeans TERBCM
TERBCM: SMSE 5.05496186, MSLL -0.6257, NLPD 2.6086 - q   0.20
TERBCM: SMSE 4.96003021, MSLL -0.8977, NLPD 2.3366 - q   0.40
TERBCM: SMSE 4.98129654, MSLL -0.9702, NLPD 2.2642 - q   0.60
TERBCM: SMSE 5.10019008, MSLL -0.9811, NLPD 2.2532 - q   0.80
TERBCM: SMSE 5.31678021, MSLL -0.9648, NLPD 2.2695 - q   1.00
TERBCM: SMSE 5.63380021, MSLL -0.9344, NLPD 2.2999 - q   1.20
TERBCM: SMSE 6.05278550, MSLL -0.8963, NLPD 2.3380 - q   1.40
TERBCM: SMSE 6.57267164, MSLL -0.8542, NLPD 2.3802 - q   1.60
TERBCM: SMSE 7.18933595, MSLL -0.8101, NLPD 2.4242 - q   1.80
TERBCM: SMSE 7.89567170, MSLL -0.7656, NLPD 2.4688 - q   2.00
TERBCM: SMSE 8.68200655, MSLL -0.7215, NLPD 2.5128 - q   2.20
TERBCM: SMSE 9.53672907, MSLL -0.6786, NLPD 2.5557 - q   2.40
TERBCM: SMSE 10.44700807, MSLL -0.6372, NLPD 2.5971 - q   2.60
TERBCM: SMSE 11.39950971, MSLL -0.5977, NLPD 2.6367 - q   2.80
TERBCM: SMSE 12.38104191, MSLL -0.5601, NLPD 2.6742 - q   3.00


### 1.1.3 Find single q

In [8]:
criterion = 'TERBCM';

In [ ]:
printf('Optimizing q\r\n');
q = grad_q(models, criterion, opts, iter=100, bs=1, lr=0.001, lambda=10, init_q=1.5);

Optimizing q
Msize =

 Columns 1 through 11:

    800    392   1617   1764   2032   1041    153    100   1366    466    887

 Columns 12 through 22:

    367      6    570    646   1259   1717    744    355   1659    217   1052

 Columns 23 through 33:

    243    635    925    415    553    937   1447   1302    742    381    608

 Columns 34 through 44:

   1623   1316    272    670     64   1921   1681    299     15    635      8

 Columns 45 through 50:

   1263    339    419    671   1302    104

Optimizing q: remain iter - 100, q - 1.5000
grad_q_norm =  0.58435
grad_q_reg = -0.46533
ttb =  4.5320
Optimizing q: remain iter - 99, q - 1.5001
grad_q_norm =  0.16302
grad_q_reg = -0.037489
ttb =  0.75676
Optimizing q: remain iter - 98, q - 1.5001
grad_q_norm =  0.21241
grad_q_reg = -0.029502
ttb =  0.69005
Optimizing q: remain iter - 97, q - 1.5001
grad_q_norm =  0.074789
grad_q_reg = -0.13283
ttb =  1.5755
Optimizing q: remain iter - 96, q - 1.5001
grad_q_norm = -1.2463
grad_q_reg = -0

Optimizing q: remain iter - 26, q - 1.5024
grad_q_norm =  0.036856
grad_q_reg = -0.0051228
ttb =  0.39122
Optimizing q: remain iter - 25, q - 1.5024
grad_q_norm =  0.41558
grad_q_reg = -0.25406
ttb =  2.2779
Optimizing q: remain iter - 24, q - 1.5024
grad_q_norm = -1.7466
grad_q_reg = -0.59135
ttb =  5.4439
Optimizing q: remain iter - 23, q - 1.5026
grad_q_norm = -1.0036
grad_q_reg = -0.20815
ttb =  2.4514
Optimizing q: remain iter - 22, q - 1.5027
grad_q_norm =  0.25758
grad_q_reg = -0.024523
ttb =  0.53662
Optimizing q: remain iter - 21, q - 1.5027
grad_q_norm =  0.20853
grad_q_reg = -0.17289
ttb =  2.0417
Optimizing q: remain iter - 20, q - 1.5027
grad_q_norm =  0.55692
grad_q_reg = -0.18667
ttb =  1.9738
Optimizing q: remain iter - 19, q - 1.5027
grad_q_norm = -0.68591
grad_q_reg = -0.33588
ttb =  3.1253
Optimizing q: remain iter - 18, q - 1.5028
grad_q_norm =    1.4600e-05
grad_q_reg =   -3.7442e-07
ttb =  0.32405
Optimizing q: remain iter - 17, q - 1.5028
grad_q_norm =  0.093900


In [ ]:
[mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

### 1.1.4 Find multi q

In [ ]:
criterion = 'TERBCM';

In [ ]:
printf('Optimizing q\r\n');
iqs = 1.01 + rand(opts.Ms,1);
qs = mul_grad_q(models, criterion, opts, iter=100, bs=1, lr=0.001, lambda=10.0, init_q=iqs);
[mu_dGP,s2_dGP,t_dGP_predict] = mul_aggregation_predict(xt,models,criterion,qs) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);

Optimizing q
Msize =

    800
    392
   1617
   1764
   2032
   1041
    153
    100
   1366
    466
    887
    367
      6
    570
    646
   1259
   1717
    744
    355
   1659
    217
   1052
    243
    635
    925
    415
    553
    937
   1447
   1302
    742
    381
    608
   1623
   1316
    272
    670
     64
   1921
   1681
    299
     15
    635
      8
   1263
    339
    419
    671
   1302
    104

Optimizing qs: remain iter - 100
qs_dist =  0.0011326
Optimizing qs: remain iter - 99
qs_dist =  0.0031373
Optimizing qs: remain iter - 98
qs_dist =  0.0035353
Optimizing qs: remain iter - 97
qs_dist =  0.0023081
Optimizing qs: remain iter - 96
qs_dist =    3.4414e-04
Optimizing qs: remain iter - 95
qs_dist =  0.0013116
Optimizing qs: remain iter - 94
qs_dist =  0.0023337
Optimizing qs: remain iter - 93
qs_dist =    7.7555e-05
Optimizing qs: remain iter - 92
qs_dist =  0.0029799
Optimizing qs: remain iter - 91
qs_dist =  0.0016430
Optimizing qs: remain iter - 90
qs_dist 

In [ ]:
qs
printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

qs =

   1.6410
   1.4818
   1.6989
   1.7803
   1.3636
   1.4261
   1.5218
   1.2537
   1.6575
   1.4568
   1.6275
   1.9896
   1.8058
   1.8817
   1.0534
   1.8600
   1.6324
   1.2197
   1.8643
   1.3426
   1.1646
   1.5326
   1.8372
   1.0394
   1.1080
   1.1025
   1.0111
   1.2748
   1.7688
   1.8632
   1.0496
   1.9530
   2.0021
   1.7972
   1.4896
   1.7294
   1.5862
   1.5562
   1.2359
   1.6317
   1.5958
   1.9430
   1.8554
   1.0728
   1.2901
   1.8338
   1.8940
   1.2020
   1.2453
   1.8570

TERBCM: SMSE 6.39103176, MSLL -0.8749, NLPD 2.3595


## 1.2 GRBCM vs TEGRBCM

### 1.2.1 GRBCM baseline

In [ ]:
criterions = {'GRBCM'}; %, 'GPoE', 'KLRBCM', 'KLGRBCM', 'GRBCM'

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    opts.partitionCriterion = partitionCriterion ;
    %[models,t_dGP_train] = aggregation_train(x,y,opts) ;
    [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
    filename = sprintf('./res/%s_%s_%s.mat', dataset, partitionCriterion, criterion);
    save(filename, 'SMSE','MSLL','NLPD')
endfor

Processing protein kmeans GRBCM
GRBCM: SMSE 5.33746421, MSLL -0.9593, NLPD 2.2750


### 1.2.2 Grid q

In [ ]:
criterions = {'TEGRBCM'}; %, 'TEGRBCM' 'TEGPoE'};
qs = 0.2:0.2:3.0;

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    opts.partitionCriterion = partitionCriterion ;
    %[models,t_dGP_train] = aggregation_train(x,y,opts) ;
    for q = qs
        [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
        [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
        printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f - q %6.2f\r\n', criterion, SMSE,MSLL,NLPD,q);
        filename = sprintf('./res/tegrbcm%s_%s_%s_%6.2f.mat', dataset, partitionCriterion, criterion, q);
        save(filename, 'SMSE','MSLL','NLPD')
    endfor
endfor

Processing protein kmeans TEGRBCM
TEGRBCM: SMSE 4.54078687, MSLL -1.0662, NLPD 2.1681 - q   0.20
TEGRBCM: SMSE 4.57666116, MSLL -1.0815, NLPD 2.1528 - q   0.40
TEGRBCM: SMSE 4.73869345, MSLL -1.0484, NLPD 2.1859 - q   0.60
TEGRBCM: SMSE 4.99478841, MSLL -1.0051, NLPD 2.2293 - q   0.80
TEGRBCM: SMSE 5.33746421, MSLL -0.9593, NLPD 2.2750 - q   1.00
TEGRBCM: SMSE 5.76315928, MSLL -0.9137, NLPD 2.3207 - q   1.20
TEGRBCM: SMSE 6.26784811, MSLL -0.8693, NLPD 2.3650 - q   1.40
TEGRBCM: SMSE 6.84575738, MSLL -0.8267, NLPD 2.4076 - q   1.60
TEGRBCM: SMSE 7.48912492, MSLL -0.7861, NLPD 2.4482 - q   1.80
TEGRBCM: SMSE 8.18844480, MSLL -0.7478, NLPD 2.4866 - q   2.00
TEGRBCM: SMSE 8.93296073, MSLL -0.7117, NLPD 2.5227 - q   2.20
TEGRBCM: SMSE 9.71126260, MSLL -0.6778, NLPD 2.5565 - q   2.40
TEGRBCM: SMSE 10.51188313, MSLL -0.6463, NLPD 2.5881 - q   2.60
TEGRBCM: SMSE 11.32382446, MSLL -0.6169, NLPD 2.6175 - q   2.80
TEGRBCM: SMSE 12.13697291, MSLL -0.5896, NLPD 2.6447 - q   3.00


### 1.2.3 Find single q

In [ ]:
criterion = 'TEGRBCM';

In [ ]:
printf('Optimizing q\r\n');
q = grad_q(models, criterion, opts, iter=100, bs=1, lr=0.01, lambda=10, init_q=1.5);

Optimizing q
Msize =

 Columns 1 through 11:

    800    392   1617   1764   2032   1041    153    100   1366    466    887

 Columns 12 through 22:

    367      6    570    646   1259   1717    744    355   1659    217   1052

 Columns 23 through 33:

    243    635    925    415    553    937   1447   1302    742    381    608

 Columns 34 through 44:

   1623   1316    272    670     64   1921   1681    299     15    635      8

 Columns 45 through 50:

   1263    339    419    671   1302    104

Optimizing q: remain iter - 100, q - 1.5000
grad_q_norm =  0.099561
grad_q_reg = -0.054484
ttb =  8.6611
Optimizing q: remain iter - 99, q - 1.5001
grad_q_norm =  0.064593
grad_q_reg = -0.038330
ttb =  8.2905
Optimizing q: remain iter - 98, q - 1.5002
grad_q_norm =  0.10067
grad_q_reg = -0.055288
ttb =  8.2046
Optimizing q: remain iter - 97, q - 1.5002
grad_q_norm =  0.32101
grad_q_reg = -0.14982
ttb =  8.4008
Optimizing q: remain iter - 96, q - 1.5005
grad_q_norm = -0.0051535
grad_q_reg =

In [ ]:
[mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

### 1.2.4 Find multi q

In [ ]:
criterion = 'TEGRBCM';

In [ ]:
printf('Optimizing q\r\n');
iqs = 1.01 + rand(opts.Ms,1);
qs = mul_grad_q(models, criterion, opts, iter=100, bs=1, lr=0.01, lambda=10, init_q=iqs);
[mu_dGP,s2_dGP,t_dGP_predict] = mul_aggregation_predict(xt,models,criterion,qs) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);

In [ ]:
qs
printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

## 1.3 GPoE vs TEGPoE

### 1.3.1 GPoE baseline

In [ ]:
criterions = {'GPoE'}; %, 'GPoE', 'KLRBCM', 'KLGRBCM', 'GRBCM'

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    opts.partitionCriterion = partitionCriterion ;
    %[models,t_dGP_train] = aggregation_train(x,y,opts) ;
    [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
    filename = sprintf('./res/%s_%s_%s.mat', dataset, partitionCriterion, criterion);
    save(filename, 'SMSE','MSLL','NLPD')
endfor

### 1.3.2 Grid q

In [ ]:
partitionCriterions = {'kmeans'}; % 'random', 'kmeans', 'knkmeans'
criterions = {'TEGPoE'}; %, 'TEGRBCM' 'TEGPoE'};
qs = 0.2:0.2:3.0;

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    opts.partitionCriterion = partitionCriterion ;
    %[models,t_dGP_train] = aggregation_train(x,y,opts) ;
    for q = qs
        [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
        [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
        printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f - q %6.2f\r\n', criterion, SMSE,MSLL,NLPD,q);
        filename = sprintf('./res/tegpoe%s_%s_%s_%6.2f.mat', dataset, partitionCriterion, criterion, q);
        save(filename, 'SMSE','MSLL','NLPD')
    endfor
endfor

### 1.3.3 Find single q

In [ ]:
criterion = 'TEGPoE';

In [ ]:
printf('Optimizing q\r\n');
q = grad_q(models, criterion, opts, iter=100, bs=1, lr=0.001, lambda=10, init_q=1.5);

In [ ]:
[mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

### 1.3.4 Find multi q

In [ ]:
printf('Optimizing q\r\n');
iqs = 1.01 + rand(opts.Ms,1);
qs = mul_grad_q(models, criterion, opts, iter=100, bs=1, lr=0.001, lambda=10, init_q=iqs);
[mu_dGP,s2_dGP,t_dGP_predict] = mul_aggregation_predict(xt,models,criterion,qs) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);

In [ ]:
qs
printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);